In [1]:
import pandas as pd
#import dnnlib
#import dnnlib.tflib as tflib
import tfutil
import dataset
import misc
import numpy as np
import config 
import tensorflow as tf
import os

import pickle
import train

resume_run_id = 4
resume_snapshot = 12000
#path = 'results/004-pgan-cifar10_wm_mullabel-cond-preset-v2-2gpus-fp32'
tfutil.init_tf(config.tf_config)
with tf.device('/gpu:0'):
    if resume_run_id is not None:
        network_pkl = misc.locate_network_pkl(resume_run_id, resume_snapshot)
        print('Loading networks from "%s"...' % network_pkl)
        G, D, Gs = misc.load_pkl(network_pkl)
            
#print(type(Gs))
#Gs.print_layers()
#dataset_args = EasyDict(tfrecord_dir='cifar10_wm_mullabel')
training_set = dataset.load_dataset(data_dir=config.data_dir, verbose=True, **config.dataset)
print('load finish')
grid_size, grid_reals, grid_labels, grid_latents = train.setup_snapshot_image_grid(G, training_set, **config.grid)

W0721 02:45:40.855454 139747280348992 deprecation_wrapper.py:119] From /tf/notebooks/progressive_growing_of_gans-master/tfutil.py:56: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0721 02:45:40.856454 139747280348992 deprecation_wrapper.py:119] From /tf/notebooks/progressive_growing_of_gans-master/tfutil.py:57: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.

W0721 02:45:40.868638 139747280348992 deprecation_wrapper.py:119] From /tf/notebooks/progressive_growing_of_gans-master/tfutil.py:65: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0721 02:45:40.869131 139747280348992 deprecation_wrapper.py:119] From /tf/notebooks/progressive_growing_of_gans-master/tfutil.py:72: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0721 02:45:42.309226 139747280348992 deprecation_wrapper.py:119] From /tf/notebooks/progressive_growing_of_

Loading networks from "results/004-pgan-cifar10_wm_mullabel-cond-preset-v2-2gpus-fp32/network-snapshot-012000.pkl"...
---226tfutil--- <module '_tfutil_network_import_module_0'> ----  G_paper ------
---226tfutil--- <module '_tfutil_network_import_module_1'> ----  D_paper ------
---226tfutil--- <module '_tfutil_network_import_module_2'> ----  G_paper ------


W0721 02:45:44.020462 139747280348992 deprecation_wrapper.py:119] From /tf/notebooks/progressive_growing_of_gans-master/dataset.py:72: The name tf.python_io.TFRecordOptions is deprecated. Please use tf.io.TFRecordOptions instead.

W0721 02:45:44.021078 139747280348992 deprecation_wrapper.py:119] From /tf/notebooks/progressive_growing_of_gans-master/dataset.py:72: The name tf.python_io.TFRecordCompressionType is deprecated. Please use tf.compat.v1.python_io.TFRecordCompressionType instead.

W0721 02:45:44.021509 139747280348992 deprecation.py:323] From /tf/notebooks/progressive_growing_of_gans-master/dataset.py:73: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
W0721 02:45:44.088004 139747280348992 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/data/util/random_seed.py:58: add_dispatch_support.<locals

Streaming data using dataset.TFRecordDataset...
---232--- dataset.TFRecordDataset ------
------214----- ['dataset', 'TFRecordDataset']
------217----- dataset
-----221---- <module 'dataset' from '/tf/notebooks/progressive_growing_of_gans-master/dataset.py'>
---226tfutil--- <module 'dataset' from '/tf/notebooks/progressive_growing_of_gans-master/dataset.py'> ----  TFRecordDataset ------
Dataset shape = [3, 32, 32]
Dynamic range = [0, 255]
Label size    = 10
load finish


In [3]:
def fp32(*values):
    if len(values) == 1 and isinstance(values[0], tuple):
        values = values[0]
    values = tuple(tf.cast(v, tf.float32) for v in values)
    return values if len(values) >= 2 else values[0]

def G_wgan_acgan(G, D, opt, training_set, minibatch_size,
    cond_weight = 1.0): # Weight of the conditioning term.

    latents = tf.random_normal([minibatch_size] + G.input_shapes[0][1:])
    labels = training_set.get_random_labels_tf(minibatch_size)
    fake_images_out = G.get_output_for(latents, labels, is_training=True)
    fake_scores_out, fake_labels_out = fp32(D.get_output_for(fake_images_out, is_training=True))
    loss = -fake_scores_out
    if D.output_shapes[1][1] > 0:
        with tf.name_scope('LabelPenalty'):
            label_penalty_fakes = tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=fake_labels_out)
        loss += label_penalty_fakes * cond_weight
    return loss


In [70]:
minibatch_size = 1024
fake_images_out = G.get_output_for(grid_latents, grid_labels, is_training=True)
fake_scores_out, fake_labels_out = fp32(D.get_output_for(fake_images_out, is_training=True))
labels = training_set.get_random_labels_tf(minibatch_size)
label_penalty_fakes = tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=fake_labels_out)
print(labels)
print(label_penalty_fakes)
print(labels.eval())

Tensor("GatherV2_17:0", shape=(1024, 10), dtype=float32)
Tensor("softmax_cross_entropy_with_logits_17/Reshape_2:0", shape=(1024,), dtype=float32)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [72]:

real_label = labels.eval()
#norm = ([labels[i]/tf.reduce_sum(labels[i]) for i in range(labels.shape[0])])
norm = labels / tf.reduce_sum(labels,axis=1)[:,None]
print(norm)
print(norm.eval())

Tensor("truediv_10251:0", shape=(1024, 10), dtype=float32)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [76]:
print(norm)
print(norm[0])
#norm = norm.eval()
for item in norm:
    count = 0
    for j in range(10):
        if item[j] > 0:
            count = count + 1
    if (count >1 ):
        print(item)
print(norm[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0.   0.   0.   0.25 0.   0.25 0.   0.   0.25 0.25]
[0.   0.   0.   0.25 0.   0.25 0.   0.   0.25 0.25]
[0.   0.   0.   0.25 0.   0.25 0.   0.   0.25 0.25]
[0.   0.   0.   0.25 0.   0.25 0.   0.   0.25 0.25]
[0.   0.   0.   0.25 0.   0.25 0.   0.   0.25 0.25]
[0.   0.   0.   0.25 0.   0.25 0.   0.   0.25 0.25]
[0.   0.   0.   0.25 0.   0.25 0.   0.   0.25 0.25]
[0.   0.   0.   0.25 0.   0.25 0.   0.   0.25 0.25]
[0.   0.   0.   0.25 0.   0.25 0.   0.   0.25 0.25]
[0.   0.   0.   0.25 0.   0.25 0.   0.   0.25 0.25]
[0.   0.   0.   0.25 0.   0.25 0.   0.   0.25 0.25]
[0.   0.   0.   0.25 0.   0.25 0.   0.   0.25 0.25]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


In [25]:
loss_acgan = label_penalty_fakes.eval()
for i in loss_acgan:
    print(i)
#print(loss_acgan)

0.09614833
0.3913411
8.302752
22.9492
28.515995
24.235977
26.995716
7.780194
9.081658
16.474438
15.329966
50.439934
9.647513
11.3658285
10.198542
7.659718
13.823521
1.0988401
13.461592
28.134516
3.2988
25.048653
18.655005
9.710426
18.500166
8.623433
15.442128
0.03475274
15.862898
14.00205
15.269704
17.054455
17.00353
9.397494
15.530214
15.67797
5.834661
24.95305
19.26633
16.013582
3.5679917
12.747533
18.258862
26.518751
11.016208
18.968521
25.935316
14.419621
28.320507
7.7169623
9.410941
19.701145
8.852641
16.190546
14.200548
0.27209592
16.656055
0.018451124
2.3843467
0.50444174
17.66817
22.147839
11.752702
26.674414
6.55802
16.231358
20.971401
14.86638
10.144613
18.773163
11.103797
7.114699
12.763834
16.518011
18.04659
0.05732846
11.460209
17.042658
3.5444078
15.760984
10.193313
19.167994
16.89057
1.3645748
15.976073
6.650551
7.764958
25.45563
12.341707
9.590899
24.942421
9.088619
16.052397
14.756618
17.893116
10.913286
0.025435133
0.049757518
22.212215
15.030834
6.947711
20.254045
9.